In [1]:
# Dependencies
from keras.preprocessing.image import ImageDataGenerator 
from keras.models import Sequential 
from keras.layers import Conv2D, MaxPooling2D 
from keras.layers import Activation, Dropout, Flatten, Dense 
from keras import backend as K 
import os
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
import numpy as np
from IPython.display import display
import random
import itertools as itt

img_width, img_height = 224, 224

In [2]:
train_data_dir = 'input/dogs-cats-images/dataset/training_set'
validation_data_dir = 'input/dogs-cats-images/dataset/test_set'
nb_train_samples =8000
nb_validation_samples = 2000
epochs = 20
batch_size = 64


In [3]:
def build_model():
    model = ResNet50(
        include_top=False,
        weights="imagenet",
        input_shape=(224, 224, 3),
        pooling='avg',
        classes=2
    )
    
    for layer in model.layers:
        layer.trainable = False
    
    output = Dense(2,activation='tanh')(model.output)
    model = Model(model.input,output)
    
    model.compile(optimizer='adam', loss='mse')
    
    return model


In [4]:
model = build_model()
model.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [5]:
model.compile(loss='binary_crossentropy', 
			optimizer='rmsprop', 
			metrics=['accuracy']) 


In [6]:
train_datagen = ImageDataGenerator( 
	rescale=1. / 255, 
	shear_range=0.2, 
	zoom_range=0.2, 
	horizontal_flip=True) 

test_datagen = ImageDataGenerator(rescale=1. / 255) 

train_generator = train_datagen.flow_from_directory( 
	train_data_dir, 
	target_size=(img_width, img_height), 
	batch_size=batch_size, 
	class_mode='binary') 

validation_generator = test_datagen.flow_from_directory( 
	validation_data_dir, 
	target_size=(img_width, img_height), 
	batch_size=batch_size, 
	class_mode='binary') 

model.fit( 
	train_generator, 
	steps_per_epoch=nb_train_samples // batch_size, 
	epochs=epochs, 
	validation_data=validation_generator, 
	validation_steps=nb_validation_samples // batch_size) 


Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Epoch 1/20
 12/125 [=>............................] - ETA: 7:42 - loss: 6.5809 - accuracy: 0.4943

KeyboardInterrupt: 

In [ ]:
model.save('model_cat_dog.h5') 

In [ ]:
# Evaluate the model using the training data
model_loss, model_accuracy = model.evaluate(validation_generator, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# Load the model
from tensorflow.keras.models import load_model
model_path = 'C:/Users/tgokh/Documents/Final-Project/model_cat_dog.h5'
modelx = load_model(model_path)

In [ ]:
from keras.models import load_model
import cv2
import numpy as np

model = load_model('model_cat_dog.h5')

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

img = cv2.imread('input/dogs-cats-images/dataset/test_set/dogs/dog.4501.jpg')
img = cv2.resize(img,(224, 224))
img = np.reshape(img,[1,224, 224,3])

classes = model.predict_classes(img)

print(classes)

In [ ]:
classes = train_generator.class_indices    
print(classes)

In [ ]:
from IPython.display import display, Image
display(Image(filename='input/dogs-cats-images/dataset/test_set/dogs/dog.4501.jpg'))

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))